In [2]:
import re
import math
import pprint as pp

In [13]:
filepath = 'data/day8.txt'

In [27]:
def read_nodes(filepath):
    with open(filepath) as file:
        data = file.read().split(' ')
    numbers = [int(n) for n in data]
    # return node and actual ending index of the node
    # node = (metadata, children_nodes)
    # return: (node, start, end)
    def get_node(start, end=None):
        try:
            total_children = numbers[start]
            total_metadata = numbers[start+1]
        except IndexError as ex:
            raise IndexError('Invalid range: {}...{}'.format(start, end))
        # no children
        if total_children == 0:
            end = (start+2)+total_metadata-1
            if end >= len(data):
                raise IndexError('Range insufficient for metadata [{}:{}] {}, {}'
                                .format(start, end, total_children, total_metadata))
            metadata = numbers[(start+2):(start+2)+total_metadata]
            return ((metadata, []), start, end)
        # for n children, n = 0..*
        # get child nodes recursively
        metadata = []
        child_nodes = []
        child_start = start+2
        for child_number in range(total_children):
            # if last children, fetch metadata and specify end
            if child_number == total_children-1:
                node, _, child_end = get_node(child_start, end)
                end = child_end+total_metadata
                metadata = numbers[child_end+1:end+1]
                child_nodes.append(node)
            # otherwise, 
            else:
                node, _, child_end = get_node(child_start, end)
                child_start = child_end+1
                child_nodes.append(node)
        return ((metadata, child_nodes), start, end)
    node, _, _ = get_node(0, len(numbers)-1)
    return node

In [28]:
nodes = read_nodes(filepath)

In [30]:
def get_total_metadata(tree):
    total_metadata = 0
    metadata, child_nodes = tree
    total_metadata += sum(metadata)
    for node in child_nodes:
        total_metadata += get_total_metadata(node)
    return total_metadata

In [36]:
total_metadata = get_total_metadata(nodes)
print(total_metadata)

41760


In [42]:
# different kind of sum
# based on the metadata which dictates how many of each of its children
# it should sum
def get_complex_total_metadata(tree):
    total_metadata = 0
    metadata, child_nodes = tree
    # if no child nodes, get sum of metadata and return
    if len(child_nodes) <= 0:
        total_metadata = sum(metadata)
        return total_metadata
    # otherwise iterate through metadata
    sum_cache = {}
    for child_index in metadata:
        child_index -= 1 # uses index system 1..*
        # ignore invalid entries
        if child_index < 0 or child_index >= len(child_nodes):
            continue
        child_node = child_nodes[child_index]
        # get the sum of the current node, while checking cache
        if child_index not in sum_cache:
            sum_cache[child_index] = get_complex_total_metadata(child_node)
        total_metadata += sum_cache[child_index]
    return total_metadata

In [43]:
complex_total_metadata = get_complex_total_metadata(nodes)
print(complex_total_metadata)

25737
